

# Public transport assignment with Optimal Strategies

In this example, we import a GTFS feed to our model, create a public transport network, create project match connectors, and perform a Spiess & Florian assignment. [Click here](https://doi.org/10.1016/0191-2615(89)90034-9) 
to check out the article.

We use data from Coquimbo, a city in La Serena Metropolitan Area in Chile.


.. admonition:: References

  * `transit_assignment`



.. seealso::
    Several functions, methods, classes and modules are used in this example:

    * :func:`aequilibrae.transit.Transit`
    * :func:`aequilibrae.transit.TransitGraphBuilder`
    * :func:`aequilibrae.paths.TransitClass`
    * :func:`aequilibrae.paths.TransitAssignment`
    * :func:`aequilibrae.matrix.AequilibraeMatrix`



In [1]:
# Imports for example construction
from uuid import uuid4
from os.path import join
from tempfile import gettempdir

from aequilibrae.transit import Transit
from aequilibrae.utils.create_example import create_example

In [2]:
# Let's create an empty project on an arbitrary folder.

##### CHANGE TEMP DIR #####
examples_dir = 'temp_examples'
#examples_dir = gettempdir()
fldr = join(examples_dir, uuid4().hex)
project = create_example(fldr, "coquimbo")

Let's create our ``Transit`` object.



In [3]:
data = Transit(project)

## Graph building
Let's build the transit network. We'll disable ``outer_stop_transfers`` and ``walking_edges`` 
because Coquimbo doesn't have any parent stations.

For the OD connections we'll use the ``overlapping_regions`` method and create some accurate line geometry later.
Creating the graph should only take a moment. By default zoning information is pulled from the project network. 
If you have your own zoning information add it using ``graph.add_zones(zones)`` then ``graph.create_graph()``. 



In [4]:
graph = data.create_graph(with_outer_stop_transfers=False, with_walking_edges=False, blocking_centroid_flows=False, connector_method="overlapping_regions")

# We drop geometry here for the sake of display.
graph.vertices.drop(columns="geometry")

,node_id,node_type,stop_id,line_id,line_seg_idx,taz_id
index,,,,,,
0,1,od,,,-1,1
1,2,od,,,-1,2
2,3,od,,,-1,3
3,4,od,,,-1,4
4,5,od,,,-1,5
...,...,...,...,...,...,...
362,363,alighting,10000000075,1_10001003000,31,
363,364,alighting,10000000076,1_10001003000,32,
364,365,alighting,10000000077,1_10001003000,33,


In [5]:
graph.vertices

,node_id,node_type,stop_id,line_id,line_seg_idx,taz_id,geometry
index,,,,,,,
0,1,od,,,-1,1,b'\x01\x01\x00\x00\x00\xf6\x98\xcd\xcb\x96\xd1...
1,2,od,,,-1,2,b'\x01\x01\x00\x00\x00\xcf\x83\x81\xb5\xdc\xce...
2,3,od,,,-1,3,"b'\x01\x01\x00\x00\x00_\xb9\xfc\x91,\xccQ\xc0\..."
3,4,od,,,-1,4,b'\x01\x01\x00\x00\x00\x1d\x07\xa1\xaa\xd0\xd4...
4,5,od,,,-1,5,b'\x01\x01\x00\x00\x00\xb5\xd3B#\xb8\xcfQ\xc0\...
...,...,...,...,...,...,...,...
362,363,alighting,10000000075,1_10001003000,31,,b'\x01\x01\x00\x00\x00:H\xef\x96\xcf\xd5Q\xc0\...
363,364,alighting,10000000076,1_10001003000,32,,"b'\x01\x01\x00\x00\x00g\x94V\xf3\xe8\xd5Q\xc0""..."
364,365,alighting,10000000077,1_10001003000,33,,b'\x01\x01\x00\x00\x00Q\xedf\xef\xfa\xd5Q\xc0\...


In [6]:
temp_df = graph.vertices
temp_df.reset_index(drop=True).drop(columns="geometry").node_type.unique()

['od', 'stop', 'boarding', 'alighting']
Categories (4, object): ['alighting', 'boarding', 'od', 'stop']

In [7]:
temp_df[temp_df.stop_id == '10000000078']

,node_id,node_type,stop_id,line_id,line_seg_idx,taz_id,geometry
index,,,,,,,
210,211,stop,10000000078,,-1,,b'\x01\x01\x00\x00\x00\xc33\xfc\x02\x15\xd6Q\x...
288,289,boarding,10000000078,1_10001003000,35,,b'\x01\x01\x00\x00\x00J\x8b&\xf9\x14\xd6Q\xc0\...
365,366,alighting,10000000078,1_10001003000,34,,b'\x01\x01\x00\x00\x00\xa8\xe1\x16\x0e\x15\xd6...


In [8]:
graph.edges

,link_id,link_type,line_id,stop_id,line_seg_idx,b_node,a_node,trav_time,freq,o_line_id,d_line_id,direction
index,,,,,,,,,,,,
0,1,on-board,1_10001001000,,0,212,290,86400.000000,inf,,,1
1,2,on-board,1_10001001000,,1,213,291,86400.000000,inf,,,1
2,3,on-board,1_10001001000,,2,214,292,86400.000000,inf,,,1
3,4,on-board,1_10001001000,,3,215,293,86400.000000,inf,,,1
4,5,on-board,1_10001001000,,4,216,294,86400.000000,inf,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...
641,642,egress_connector,,,-1,166,112,630.543431,inf,,,1
642,643,egress_connector,,,-1,177,112,754.081137,inf,,,1
643,644,egress_connector,,,-1,178,112,386.262027,inf,,,1


In [9]:
graph.edges.link_type.unique()

['on-board', 'boarding', 'alighting', 'dwell', 'access_connector', 'egress_connector']
Categories (6, object): ['access_connector', 'alighting', 'boarding', 'dwell', 'egress_connector', 'on-board']

The graphs also also stored in the ``Transit.graphs`` dictionary. They are keyed by the 'period_id' they 
were created for. A graph for a different 'period_id' can be created by providing ``period_id=`` in the 
``Transit.create_graph`` call. You can view previously created periods with the ``Periods`` object.



In [10]:
periods = project.network.periods
periods.data

,period_id,period_start,period_end,period_description
0,1,0,86400,"Default time period, whole day"


## Connector project matching



In [11]:
project.network.build_graphs()

Now we'll create the line strings for the access connectors, this step is optinal but provides more accurate distance 
estimations and better looking geometry.

Because Coquimbo doesn't have many walking edges we'll match onto the ``"c"`` graph.



In [12]:
project.network.graphs

{'b': <aequilibrae.paths.graph.Graph at 0x24990bb9a60>,
 'c': <aequilibrae.paths.graph.Graph at 0x24990bb9880>,
 't': <aequilibrae.paths.graph.Graph at 0x24990bcb160>,
 'w': <aequilibrae.paths.graph.Graph at 0x24991f27070>}

In [13]:
# c for All motorized vehicles
graph.create_line_geometry(method="connector project match", graph="c")

c:\Users\germa\Documents\UQ\outer_loop\aequilibrae\aequilibrae\transit\transit_graph_builder.py:1215: UserWarning: In its current implementation, the "connector project match" method may take a while for large networks.
  warnings.warn(


## Saving and reloading
Lets save all graphs to the 'public_transport.sqlite' database.



In [24]:
data.save_graphs()

c:\Users\germa\Documents\UQ\outer_loop\aequilibrae\aequilibrae\transit\transit.py:99: UserWarning: Currently only a single transit graph can be saved and reloaded. Multiple graph support is plan for a future release.
  warnings.warn(


We can reload the saved graphs with ``data.load``. 
This will create new ``TransitGraphBuilder``\'s based on the 'period_id' of the saved graphs.
The graph configuration is stored in the 'transit_graph_config' table in 'project_database.sqlite' 
as serialised JSON.



In [25]:
data.load()

c:\Users\germa\Documents\UQ\outer_loop\aequilibrae\aequilibrae\transit\transit.py:113: UserWarning: Currently only a single transit graph can be saved and reloaded. Multiple graph support is plan for a future release. `period_ids` argument is currently ignored.
  warnings.warn(


Links and nodes are stored in a similar manner to the 'project_database.sqlite' database.

## Reading back into AequilibraE
You can create back in a particular graph via it's 'period_id'.



In [30]:
from aequilibrae.project.database_connection import database_connection
from aequilibrae.transit.transit_graph_builder import TransitGraphBuilder

In [31]:
pt_con = database_connection("transit")

graph_db = TransitGraphBuilder.from_db(pt_con, periods.default_period.period_id)
graph_db.vertices.drop(columns="geometry")

,node_id,node_type,stop_id,line_id,line_seg_idx,taz_id
0,1,od,,,-1,1
1,2,od,,,-1,2
2,3,od,,,-1,3
3,4,od,,,-1,4
4,5,od,,,-1,5
...,...,...,...,...,...,...
362,363,alighting,10000000075,1_10001003000,31,
363,364,alighting,10000000076,1_10001003000,32,
364,365,alighting,10000000077,1_10001003000,33,
365,366,alighting,10000000078,1_10001003000,34,


In [32]:
graph_db.edges

,link_id,link_type,line_id,stop_id,line_seg_idx,b_node,a_node,trav_time,freq,o_line_id,d_line_id,direction
0,1,on-board,1_10001001000,,0,212,290,86400.000000,inf,,,1
1,2,on-board,1_10001001000,,1,213,291,86400.000000,inf,,,1
2,3,on-board,1_10001001000,,2,214,292,86400.000000,inf,,,1
3,4,on-board,1_10001001000,,3,215,293,86400.000000,inf,,,1
4,5,on-board,1_10001001000,,4,216,294,86400.000000,inf,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...
641,642,egress_connector,,,-1,166,112,630.543431,inf,,,1
642,643,egress_connector,,,-1,177,112,754.081137,inf,,,1
643,644,egress_connector,,,-1,178,112,386.262027,inf,,,1
644,645,egress_connector,,,-1,179,112,558.108038,inf,,,1


## Converting to a AequilibraE graph object
To perform an assignment we need to convert the graph builder into a graph.



In [16]:
transit_graph = graph.to_transit_graph()

In [17]:
transit_graph

## Mock demand matrix
We'll create a mock demand matrix with demand 1 for every zone.
We'll also need to convert from ``zone_id``\'s to ``node_id``\'s.



In [18]:
import numpy as np
from aequilibrae.matrix import AequilibraeMatrix

In [19]:
zones_in_the_model = len(transit_graph.centroids)

names_list = ['pt']

mat = AequilibraeMatrix()
mat.create_empty(zones=zones_in_the_model,
                 matrix_names=names_list,
                 memory_only=True)
mat.index = transit_graph.centroids[:]
mat.matrices[:, :, 0] = np.full((zones_in_the_model, zones_in_the_model), 1.0)
mat.computational_view()

In [29]:
mat.get_matrix('pt').shape

(133, 133)

In [28]:
mat.get_matrix('pt')

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

## Hyperpath generation/assignment
We'll create a ``TransitAssignment`` object as well as a ``TransitClass``



In [20]:
from aequilibrae.paths import TransitAssignment, TransitClass

In [21]:
# Create the assignment class
assigclass = TransitClass(name="pt", graph=transit_graph, matrix=mat)

assig = TransitAssignment()

assig.add_class(assigclass)

# We need to tell AequilbraE where to find the appropriate fields we want to use,  
# as well as the assignment algorithm to use.
assig.set_time_field("trav_time")
assig.set_frequency_field("freq")

assig.set_algorithm("os")

# When there's multiple matrix cores we'll also need to set the core to use for the demand.
assigclass.set_demand_matrix_core("pt")

# Let's perform the assignment for the transit classes added
assig.execute()


View the results



In [40]:
assig.classes[0].graph
for cls in assig.classes:
    print(cls._id)
    print(cls.graph)
    print(cls.matrix)

pt


In [39]:
df_graph = cls.graph.graph
df_graph

,link_id,a_node,b_node,direction,id,link_type,line_id,stop_id,line_seg_idx,trav_time,freq,o_line_id,d_line_id,geometry,__supernet_id__,__compressed_id__
0,479,18,158,1,0,egress_connector,,,-1,2146.077896,inf,,,b'\x01\x02\x00\x00\x00S\x00\x00\x00[S\xea\n\x8...,478,0
1,480,18,159,1,1,egress_connector,,,-1,632.392145,inf,,,"b""\x01\x02\x00\x00\x00!\x00\x00\x00[S\xea\n\x8...",479,1
2,481,18,183,1,2,egress_connector,,,-1,269.031753,inf,,,"b""\x01\x02\x00\x00\x00\x0e\x00\x00\x00[S\xea\n...",480,2
3,482,19,158,1,3,egress_connector,,,-1,1240.173389,inf,,,"b""\x01\x02\x00\x00\x00$\x00\x00\x00\x82\xa7\xe...",481,3
4,483,19,183,1,4,egress_connector,,,-1,2174.016494,inf,,,b'\x01\x02\x00\x00\x00V\x00\x00\x00\x82\xa7\xe...,482,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,74,362,284,1,641,on-board,1_10001003000,,31,86400.000000,inf,,,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00:H\xef\x...,73,638
642,75,363,285,1,642,on-board,1_10001003000,,32,86400.000000,inf,,,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00g\x94V\x...,74,639
643,76,364,286,1,643,on-board,1_10001003000,,33,86400.000000,inf,,,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00Q\xedf\x...,75,640
644,77,365,287,1,644,on-board,1_10001003000,,34,86400.000000,inf,,,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xa8\xe1...,76,641


In [26]:
assigclass.matrix

In [22]:
assig.results()

,pt_volume
479,238.0
480,18.0
481,24.0
482,18.0
483,12.0
...,...
74,0.0
75,0.0
76,0.0
77,0.0


## Saving results
We'll be saving the results to another sqlite db called 'results_database.sqlite'. 
The 'results' table with 'project_database.sqlite' contains some metadata about each table in 
'results_database.sqlite'.



In [79]:
assig.save_results(table_name='hyperpath example')

Wrapping up



In [80]:
project.close()